# <font color=black> Figure 1 Spinal cord morphometry </font>
<hr style="border:1px solid black">

### Imports

In [ ]:
import sys,json
import glob, os
import pandas as pd
import numpy as np
from palettable.colorbrewer.sequential import GnBu_9
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

#statistics
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

main_dir="/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project/2025_brsc_aging_project"
sys.path.append(main_dir + "/code/")
from sc_structural_analyses import StructuralMetrics

from plotting import Plotting
import matplotlib.pyplot as plt

from brsc_statistics import Statistics 
from scipy.stats import spearmanr
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Load config file ------------------------------------------------------------
config_file=main_dir + '/config/analyses/brsc_structural.json'
with open(config_file) as config_f: # the notebook should be in 'xx/notebook/' folder #config_proprio
    config = json.load(config_f) # load config file should be open first and the path inside modified

plot=Plotting(config_file,"test")
stat_func=Statistics(config=config,ana_dir="",analysis="")

# atlas_labels ------------------------------
atlas_labels_f=glob.glob(config["project_dir"] + config['template']["spinalcord"]['tracts_atlas'].split(".")[0] + ".txt")[0]
atlas_labels=np.genfromtxt(atlas_labels_f, usecols=0,skip_header=1, dtype="S", delimiter="\t", encoding=None)
atlas_labels = np.array([label.decode("utf-8") for label in atlas_labels])

#initial sub_metrics info


<hr style="border:1px solid black">

## <font color=#0B7CC3> A. Extract features

In [ ]:
sub_metrics={"MTR":["MTR"],"DWI":["FA","RD","AD","MD"]}
atlas_labels=['dorsal_col', 'ventral_funi','lat_funi']
#sub_metrics={"MTR":["MTR_wm"]}

df_metrics={}
atlas_f=[]

for contrast in sub_metrics:
    df_metrics[contrast]={}
    mask_path={}
    for sub_metric in sub_metrics[contrast]:
        print(sub_metric)
        files_indivspace=[]
        #config["participants_IDs_" + contrast]=["A003"]
        if len(sub_metrics[contrast])>1:
            metrics=StructuralMetrics(config,contrast=contrast,metric=sub_metric) # initialize the function
        else:
            metrics=StructuralMetrics(config,contrast=contrast)
        atlas_f=[]
        for ID_nb, ID in enumerate(config["participants_IDs_" + contrast]):
            preproc_dir= config["preprocess_dir"]["bmpd"] if ID[0]=="P" else config["preprocess_dir"]["stratals"]
            if contrast=="T2s":
                files_indivspace.append(glob.glob(preproc_dir+ config["indiv_space"][contrast].format(ID))[0])
                if sub_metric=="T2s_gm" or sub_metric=="T2s_wm":
                    atlas_f.append(preproc_dir + config["indiv_space"][sub_metric + "_tracts_atlas"].format(ID))
                    files_indivspace.append(glob.glob(preproc_dir+ config["indiv_space"][contrast].format(ID))[0])
                
                else:
                    atlas_f.append(preproc_dir + config["indiv_space"][contrast+ "_atlas"].format(ID))
                    files_indivspace.append(glob.glob(preproc_dir+ config["indiv_space"][sub_metric].format(ID))[0])
                
            elif contrast=="MTR":
                files_indivspace.append(glob.glob(preproc_dir+ config["indiv_space"][contrast].format(ID))[0])
                atlas_f.append(preproc_dir + config["indiv_space"][sub_metric +  "_tracts_atlas"].format(ID))

                
            elif contrast=="DWI":
                files_indivspace.append(glob.glob(preproc_dir+ config["indiv_space"][sub_metric].format(ID))[0])
                atlas_f.append(preproc_dir + config["indiv_space"][contrast + "_tracts_atlas"].format(ID))


        df_metrics[contrast][sub_metric]=metrics.extract_metric_rois(IDs=config["participants_IDs_" + contrast],
                            input_f=files_indivspace,
                            atlas_f=atlas_f,
                            atlas_labels=atlas_labels,
                            metric=sub_metric,
                            measure="mean",
                            space="indiv_space",
                            tag="_tracts"  ,norm=False,redo=False,verbose=1)

    
       

<hr style="border:1px solid black">

## <font color=#0B7CC3> D. Age and sex effects


In [ ]:
sub_metrics={"MTR":["MTR"],"DWI":["FA","RD","AD","MD"]}

df_mean_metrics={};
mean_results = [];roi_results=[];

for contrast in sub_metrics:
    df_mean_metrics[contrast]={}
    for sub_metric in sub_metrics[contrast]:
        print(sub_metric)
        y = sub_metric
        signed_r2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,ci_r2_age,  ci_beta_age, ci_beta_cov =stat_func.regression_model(df=df_metrics[contrast][sub_metric],y=y,predictor="age",covariates=["sex"],random='IDs')
        mean_results.append({"contrast": contrast,"sub_metric": y,"signed_r2": signed_r2,"p_age": p_age,"p_sex": p_sex,"beta_age": beta_age,"beta_sex": beta_sex,"tvalue_age": stat_age,"tvalue_sex": stat_sex})

        for roi in np.unique(df_metrics[contrast][sub_metric]["rois"]):
            df_roi_metrics=df_metrics[contrast][sub_metric][df_metrics[contrast][sub_metric]["rois"]==roi]
            signed_r2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,ci_r2_age,  ci_beta_age, ci_beta_cov =stat_func.regression_model(df=df_roi_metrics,y=y,predictor="age",covariates=["sex"])
            roi_results.append({"contrast": contrast,"sub_metric": sub_metric,"rois":roi,"signed_r2": signed_r2,"p_age": p_age,"p_sex": p_sex,"beta_age": beta_age,"beta_sex": beta_sex,"tvalue_age": stat_age,"tvalue_sex": stat_sex})

# Convert to DataFrame
mean_result_df= pd.DataFrame(mean_results)
roi_result_df=pd.DataFrame(roi_results)

for roi in np.unique(df_metrics[contrast][sub_metric]["rois"]):
    print(roi + " absolute mean t-value: " + str(np.round(np.mean(np.abs(roi_result_df[(roi_result_df["rois"]==roi)
                  &(roi_result_df["sub_metric"].isin(["MTR_tracts","T2s_vx_gm","T2s_vx_wm","FA","RD","AD","MD"]))]["tvalue_age"])),3)) + " ± " +
         str(np.round(np.std(np.abs(roi_result_df[(roi_result_df["rois"]==roi)
                  &(roi_result_df["sub_metric"].isin(["MTR_tracts","T2s_vx_gm","T2s_vx_wm","FA","RD","AD","MD"]))]["tvalue_age"])),2)))


In [ ]:
from statsmodels.stats.multitest import multipletests

# Step 1: extract p-values
p_age = roi_result_df["p_age"].values
p_sex = roi_result_df["p_sex"].values

# Step 2: apply FDR correction
reject, page_corrected, _, _ = multipletests(p_age, method='fdr_bh')
reject, psex_corrected, _, _ = multipletests(p_sex, method='fdr_bh')

# Step 3: add results to your DataFrame
roi_result_df["page_fdr"] = page_corrected
roi_result_df["psex_fdr"] = psex_corrected
roi_result_df

In [ ]:
color=["#3AA198"]#,"#3AA198","#68BCB4","#86C9C3","#C3E4E1",
output_dir=config["project_dir"] +  "/figures/f01_structural/preprint2025/figures/"
for contrast in sub_metrics:
    print(contrast)
    color=["#3AA198"] if contrast=="MTR" else ["#C7093F"]
    
    for sub_metric in sub_metrics[contrast]:
        for roi in np.unique(df_metrics[contrast][sub_metric]["rois"]):
            print(roi)
            sub_df=df_metrics[contrast][sub_metric][df_metrics[contrast][sub_metric]["rois"]==roi]
            plot.lmplots(df=sub_df,color=color,
                         x_data="age",
                         y_data=sub_metric,xmin=15,xmax=85,
                     indiv_values=True,
                         output_dir=output_dir,output_tag=sub_metric + "_" +roi+"_WMtract",
                         save=True)